# This jupyter notebook contains examples of
- some basic functions related to native contacts / bias contact analyses
- interactive plots

In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import MDAnalysis as mda
import pyrexMD.core as core
import pyrexMD.misc as misc
import pyrexMD.topology as top
import pyrexMD.analysis.analysis as ana
import pyrexMD.analysis.contacts as con

# Native Contact Analyses
First we have to define MDAnalysis universes to handle data. In this case we just want to compare two different conformations which have no trajectory data.

In [ ]:
pdb1 = "files/contacts/1vii_ref.pdb"
pdb2 = "files/contacts/1vii_unfolded_frame.pdb"

u1 = mda.Universe(pdb1)
u2 = mda.Universe(pdb2)

We can check the models/conformations with the trajectory viewer.

In [ ]:
tv1 = core.iPlayer(u1)
tv1()

tv2 = core.iPlayer(u2)
tv2()

We can easily obtain a list with native contacts and save them to a log file. Here we use a cutoff distance of 6 Anström and calculate native contacts for the selection string "protein". The function returns:

- NC: native contacts (unique RES pairs)
- NC_d: detailed list of NCs containing (RES pairs), (ATOM numbers), (ATOM names)


In [ ]:
NC, NC_d = con.get_Native_Contacts(u1, d_cutoff=6.0, sel="protein", save_as="files/contacts/1vii_native_contacts.txt")

Quick look at the content:

In [ ]:
NC

In [ ]:
NC_d

The contact map of the folded reference structure can be obtained via

In [ ]:
fig, ax = con.plot_Contact_Map(u1, sel="protein")

We can also compare the native contacts of a model (grey) with contacts from an input file (DCA_fin) and test if they are native (green) or non-native (red) within the model.

In [ ]:
fig, ax = con.plot_Contact_Map(u1, DCA_fin="files/contacts/1vii_mixed_contacts.txt", n_DCA=None)

By extracting the contacts from any model and using them as input for another model, we can use the contact map  plot to compare which contacts remained native (green),  which broke up (red) and which were newly formed (grey).

As an example we compare the contacts between the unfolded and folded state:

In [ ]:
_ = con.get_Native_Contacts(u2, d_cutoff=6.0, sel="protein", save_as="files/contacts/1vii_unfolded_contacts.txt")
fig, ax = con.plot_Contact_Map(u1, DCA_fin="files/contacts/1vii_unfolded_contacts.txt", n_DCA=None)

In a similar way, we can get the native contact distances. The function returns:

- NC: native contacts
- NC_dist: native contact distances
- DM: distance matrices


In [ ]:
NC, NC_dist, DM = con.get_NC_distances(u1, u1, sel="protein", d_cutoff=6.0)

We can use the print_table function to quickly take a look on the content. The table only prints a certain number of entries (defaults to 30).

In [ ]:
misc.cprint("distance  NC pair", "blue")
_ = misc.print_table([NC_dist, NC], spacing=10, verbose_stop=10)

In a similar way, we can also create contact maps and display the contact distances.

In [ ]:
fig, ax = con.plot_Contact_Map_Distances(u1, NC, NC_dist, pdbid="1vii", sel="protein")

Simulations are often biased using contacts from theoretical or experimental sources. We can check the contacts for their True Positive Rate (TPR) based on their number of ranked contacts.

For example:

In [ ]:
fig, ax = con.plot_DCA_TPR(u1, DCA_fin="files/contacts/1vii.rr", n_DCA=80, d_cutoff=8.0)

The input file '1vii.rr' contains predicted contacts with cutoff = 8A.

The figure shows:
- blue line: TPR
- red line: 75% cutoff threshold (TPR of used number of contacts should be above 75% for contact-guided REX, see  https://doi.org/10.1371/journal.pone.0242072)
- orange lines: suggested/guessed optimum number of contacts and the corresponding TPR
- orange region: suggested region of interest between L/2 and L contacts


# QNative Analysis

The fraction of realized native contacts (QValues, or specifically "QNative") is often of great interest.

Exemplarily, we use a very short simulation and define the reference structure and the mobile structure (which contains the trajectory data).

In [ ]:
ref = "files/traj/2hba_ChainB_ref.pdb"
top = "files/traj/traj_protein.tpr"
traj = "files/traj/traj_protein.xtc"
u1 = mda.Universe(top,traj)   # mobile
u2 = mda.Universe(ref)        # reference

tv1 = core.iPlayer(u1)
tv1()
tv2 = core.iPlayer(u2)
tv2()

We calculate QNative. The function returns:

- FRAMES: array with frames
- QNATIVE: array with fraction of native contacts

In [ ]:
FRAMES, QNATIVE = con.get_QNative(u1,u2, method="radius_cut", sel="protein and name CA", marker="None", lw=1)

Many analysis functions also generate plots automatically but can be supressed via plot=False.



To obtain the list of contact pairs used during the QNative analysis, we can use the function get_Native_Contacts as before.

The function returns:
- NC: native contacts (unique RES pairs)
- NC_d: detailed list of NCs containing (RES pairs), (ATOM numbers), (ATOM names)

In [ ]:
NC, NC_d = con.get_Native_Contacts(u1, sel="protein and name CA")
NC

# QBias Analysis

Analogously to the QNative analysis we can look at the fraction of formed bias contacts ("QBias"). The first steps are identical to before:

In [ ]:
ref = "files/traj/2hba_ChainB_ref.pdb"
top = "files/traj/traj_protein.tpr"
traj = "files/traj/traj_protein.xtc"
u1 = mda.Universe(top,traj)   # mobile
u2 = mda.Universe(ref)        # reference

sel = "protein and name CA"
NC, NC_d = con.get_Native_Contacts(u2, sel=sel)

Next we calculate QBias and directly plot the time evolution. The function returns:

- FRAMES: array with frame numbers
- QBIAS: array with fraction of formed bias contacts
- CM: array with contact matrices

In [ ]:
FRAMES, QBIAS, CM = con.get_QBias(u1, NC, d_cutoff=6.0, warn=False, marker=None, lw=1)

We can test the functionality by using native contacts as bias contacts compare the difference between QNative and QBias.

In [ ]:
misc.cprint("Comparison: QNative - QBias ; include_selfcontacts=False", "blue")
print(QNATIVE - QBIAS)

Note: the qvalue algorithm for native contacts ("QNative") includes selfcontacts. Comparison of both methods yields better results when include_selfcontacts (bool, see kwargs) is set to True. However this improves the calculated QBias value artificially (e.g. even when all used bias contacts are never formed, QBias will not be zero due to the selfcontact counts). Hence include_selfcontacts defaults to False.

In [ ]:
#FRAMES, QBIAS, CM = con.get_QBias(u1, NC, d_cutoff=6.0, include_selfcontacts=True, warn=False, marker=None, lw=1)
FRAMES, QBIAS, CM = con.get_QBias(u1, NC, d_cutoff=6.0, include_selfcontacts=True, warn=False, plot=False)

misc.cprint("Comparison: QNative - QBias ; include_selfcontacts=True", "blue")
print(QNATIVE - QBIAS)

We can extract formed bias contacts for individual frames (here frame 0) via:

In [ ]:
CP = con.get_formed_contactpairs(u1, CM[0], include_selfcontacts=False)
CP

# Interactive Plots

Short showcase of the interactive plot function. It links the trajectory viewer to a 2D plot, where you can use inspect conformations at specific xy-values.


- ctrl-click or shift-click into plot to jump to specific frames
- click on TIME/FRAME widget to toggle between modes
- modify the representation via UI

In [ ]:
IP = core.iPlot(u1, xdata=FRAMES, ydata=QBIAS, xlabel="frame", ylabel="QBias")
IP()